In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import shuffle, resample

## Load data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news = pd.read_csv("/content/drive/MyDrive/Masterarbeit/Crawler/data/FinalDataset/Ablation01/Stepwords_CleanData_FinalDataset_ForModel_Ablation1.csv", index_col=0)

In [ ]:
# Get the same train and test data
def split_df_in_train_test(df):
    df = df.reset_index()
    split_point = int(np.round(df.shape[0]) * 0.8)
    df_train = df.loc[:split_point-1,:]
    df_test = df.loc[split_point:,:]
    return df_train, df_test

In [ ]:
n_samples = 6000

In [ ]:
left_FE = resample(shuffle(news[(news["label"]=="links") & (news["Länge"]<512)], random_state=42), \
         random_state=42, n_samples=n_samples)
center_FE = resample(shuffle(news[(news["label"]=="central") & (news["Länge"]<512)], random_state=42), \
         random_state=42, n_samples=n_samples)
right_FE = resample(shuffle(news[(news["label"]=="rechts") & (news["Länge"]<512)], random_state=42), \
         random_state=42, n_samples=n_samples)

In [ ]:
train = pd.concat([split_df_in_train_test(left_FE)[0], \
    split_df_in_train_test(center_FE)[0], \
    split_df_in_train_test(right_FE)[0]])

In [ ]:
test =  pd.concat([split_df_in_train_test(left_FE)[1], \
    split_df_in_train_test(center_FE)[1], \
    split_df_in_train_test(right_FE)[1]])

In [ ]:
X_train, y_train = train["NeuGenerierterText_str"], train["label"]
X_test, y_test = test["NeuGenerierterText_str"], test["label"]

# Creating German Stopwords

In [ ]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32919 sha256=b6cf7541721c0fb9fd085658f7a7da97f32b4c16eca60877c69dbe36cd3ffca2
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
from stop_words import get_stop_words

#stop_words = get_stop_words('de')
stop_words = get_stop_words('german')

## Generate TFIDF vector

### Bigram:

In [ ]:
bigram_vec = TfidfVectorizer(stop_words=stop_words, max_features=30000, ngram_range=(1, 2))

X_train_bi = bigram_vec.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_test_bi = bigram_vec.transform(X_test.apply(lambda x: np.str_(x)))

### Trigram

In [ ]:
trigram_vec = TfidfVectorizer(stop_words=stop_words, max_features=30000, ngram_range=(1, 3))

X_train_tri = trigram_vec.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_test_tri = trigram_vec.transform(X_test.apply(lambda x: np.str_(x)))

## Generate LabelEncoder

In [ ]:
label_enc = LabelEncoder()
y_train_enc = label_enc.fit_transform(y_train)
y_test_enc = label_enc.fit_transform(y_test)

In [ ]:
label_enc.inverse_transform([0, 1, 2]) 

array(['central', 'links', 'rechts'], dtype=object)

In [ ]:
label = [0, 1, 2]
target_label = ['central', 'links', 'rechts']

## Apply classifiers

In [ ]:
def run_classifier(clf, X_train, X_test, y_train, y_test, label, target_label):
    
    print("Training of the classifier: {} \n".format(clf))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print("\n")

    print("Accuracy of the classifier:     ")
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)

    print("\n")

    print("Confusion Matrix of the classifier: \n")
    con_mat = confusion_matrix(y_test, y_pred, labels=label)
    print(con_mat)

    print("\n")

    print("Classification Report of the classifier: \n")
    report = classification_report(y_test, y_pred, target_names=target_label)
    print(report)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
svc = LinearSVC()
lr = LogisticRegression(multi_class="multinomial", solver="saga")
nb = BernoulliNB()

### ...using with bigrams

#### Decision Tree

In [ ]:
run_classifier(dt, X_train_bi, X_test_bi, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best') 



Accuracy of the classifier:     
0.7016666666666667


Confusion Matrix of the classifier: 

[[882 134 184]
 [144 883 173]
 [233 206 761]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.70      0.73      0.72      1200
       links       0.72      0.74      0.73      1200
      rechts       0.68      0.63      0.66      1200

    accuracy                           0.70      3600
   macro avg       0.70      0.70      0.70      3600
weighted avg       0.70      0.70 

#### Naive Bayes

In [ ]:
run_classifier(nb, X_train_bi, X_test_bi, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 



Accuracy of the classifier:     
0.6463888888888889


Confusion Matrix of the classifier: 

[[730  69 401]
 [150 718 332]
 [219 102 879]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.66      0.61      0.64      1200
       links       0.81      0.60      0.69      1200
      rechts       0.55      0.73      0.63      1200

    accuracy                           0.65      3600
   macro avg       0.67      0.65      0.65      3600
weighted avg       0.67      0.65      0.65      3600



#### Support Vector Machine

In [ ]:
run_classifier(svc, X_train_bi, X_test_bi, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0) 



Accuracy of the classifier:     
0.7705555555555555


Confusion Matrix of the classifier: 

[[953  98 149]
 [125 955 120]
 [184 150 866]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.76      0.79      0.77      1200
       links       0.79      0.80      0.79      1200
      rechts       0.76      0.72      0.74      1200

    accuracy                           0.77      3600
   macro avg       0.77      0.77      0.77      3600
weighted avg       0.77      0.77      0.77      3600



#### Logistic Regression

In [ ]:
run_classifier(lr, X_train_bi, X_test_bi, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False) 



Accuracy of the classifier:     
0.7527777777777778


Confusion Matrix of the classifier: 

[[940 116 144]
 [154 911 135]
 [195 146 859]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.73      0.78      0.76      1200
       links       0.78      0.76      0.77      1200
      rechts       0.75      0.72      0.73      1200

    accuracy                           0.75      3600
   macro avg       0.75      0.75      0.75      3600
weighted avg       0.75      0.75      0.75      3600



### ...using trigrams

#### Decision Tree

In [ ]:
run_classifier(dt, X_train_tri, X_test_tri, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best') 



Accuracy of the classifier:     
0.7141666666666666


Confusion Matrix of the classifier: 

[[903 114 183]
 [154 885 161]
 [230 187 783]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.70      0.75      0.73      1200
       links       0.75      0.74      0.74      1200
      rechts       0.69      0.65      0.67      1200

    accuracy                           0.71      3600
   macro avg       0.71      0.71      0.71      3600
weighted avg       0.71      0.71 

#### Naive Bayes

In [ ]:
run_classifier(nb, X_train_tri, X_test_tri, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 



Accuracy of the classifier:     
0.6330555555555556


Confusion Matrix of the classifier: 

[[741  39 420]
 [170 643 387]
 [229  76 895]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.65      0.62      0.63      1200
       links       0.85      0.54      0.66      1200
      rechts       0.53      0.75      0.62      1200

    accuracy                           0.63      3600
   macro avg       0.67      0.63      0.64      3600
weighted avg       0.67      0.63      0.64      3600



#### Support Vector Machine

In [ ]:
run_classifier(svc, X_train_tri, X_test_tri, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0) 



Accuracy of the classifier:     
0.7741666666666667


Confusion Matrix of the classifier: 

[[960 100 140]
 [121 955 124]
 [180 148 872]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.76      0.80      0.78      1200
       links       0.79      0.80      0.79      1200
      rechts       0.77      0.73      0.75      1200

    accuracy                           0.77      3600
   macro avg       0.77      0.77      0.77      3600
weighted avg       0.77      0.77      0.77      3600



#### Logistic Regression

In [ ]:
run_classifier(lr, X_train_tri, X_test_tri, y_train_enc, y_test_enc, label, target_label)

Training of the classifier: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False) 



Accuracy of the classifier:     
0.7533333333333333


Confusion Matrix of the classifier: 

[[945 113 142]
 [156 913 131]
 [196 150 854]]


Classification Report of the classifier: 

              precision    recall  f1-score   support

     central       0.73      0.79      0.76      1200
       links       0.78      0.76      0.77      1200
      rechts       0.76      0.71      0.73      1200

    accuracy                           0.75      3600
   macro avg       0.75      0.75      0.75      3600
weighted avg       0.75      0.75      0.75      3600

